In [2]:
import requests
from requests import Response
from pydantic import RootModel, BaseModel, Field, field_validator
from datetime import datetime
import pandas as pd

class Site(BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量', '可還數量', mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]
    
try:
    youbike_url:Response = requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,1,27,2024-07-01 19:41:21
1,大安區,21,11,10,2024-07-01 19:43:20
2,大安區,16,8,8,2024-07-01 19:44:20
3,大安區,11,3,8,2024-07-01 19:44:20
4,大安區,16,1,15,2024-07-01 19:44:20
...,...,...,...,...,...
1424,臺大公館校區,30,0,29,2024-07-01 19:43:20
1425,臺大公館校區,20,0,20,2024-07-01 19:31:15
1426,臺大公館校區,24,8,15,2024-07-01 19:39:21
1427,臺大公館校區,40,27,4,2024-07-01 19:44:20


In [7]:
df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg([('車輛數', 'sum'), ('站點數', 'count')])


總數量       可借數量       可還數量     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
行政區域                                   
中山區     3819  156  1566  156  2230  156
中正區     3209  121  1472  121  1710  121
信義區     3331  106  1293  106  1938  106
內湖區     3499  168  1416  168  2046  168
北投區     2472   94   950   94  1492   94
南港區     2317   92  1121   92  1176   92
士林區     2937  127  1076  127  1799  127
大同區     1629   62   608   62  1012   62
大安區     4987  185  2082  185  2818  185
文山區     2203   97   865   97  1335   97
松山區     2343   88   899   88  1416   88
臺大公館校區  1437   53   496   53   904   53
萬華區     2057   80  1031   80  1005   80

In [10]:
# pivot
# stack, unstack
# 欄位有名稱
df.stack(future_stack=True)

0     行政區域                    大安區
      總數量                      28
      可借數量                      1
      可還數量                     27
      時間      2024-07-01 19:41:21
                     ...         
1428  行政區域                 臺大公館校區
      總數量                      18
      可借數量                      0
      可還數量                     18
      時間      2024-07-01 19:39:16
Length: 7145, dtype: object